In [ ]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Simple conversation Tool

In [2]:
from smolagents import Tool
from openai import OpenAI
class Simple_Conversation_Tool(Tool):
    """
    A tiny small-talk tool that delegates response crafting to OpenAI.
    Great for greetings, pleasantries, and casual chit-chat.
    """
    name = "Simple_Conversation_Tool"
    description = (
        "Handles simple conversational messages like greetings and small talk. "
        "Use this tool when the user says hi/hello/hey/good morning, asks how you are, "
        "thanks you, or otherwise initiates casual chat."
    )

    inputs = {
        "message": {"type": "string", "description": "The user's chat message."}
    }

    output_type = "string"

    def __init__(self, api_key: str, model_id: str = "gpt-4o-mini"):
        super().__init__()
        self.client = OpenAI(api_key=api_key)
        self.model_id = model_id
        self.system_prompt = (
            "You are a friendly, concise customer support greeter. "
            "Respond in one or two sentences maximum. "
            "Be warm, helpful, and professional. "
            "If the user asks for support beyond greetings, invite them to share details."
        )

    def forward(self, message: str) -> str:
        # Lightweight guardrail: only handle small-talk; let the agent do other stuff
        # smalltalk_triggers = (
        #     "hi", "hello", "hey", "good morning", "good afternoon",
        #     "good evening", "how are you", "what's up", "sup", "salam",
        #     "assalam", "thanks", "thank you"
        # )
        msg_lower = message.lower()
        # if not any(t in msg_lower for t in smalltalk_triggers):
        #     return (
        #         "I'm here for quick greetings and small talk. "
        #         "Tell me a bit more about what you need help with!"
        #     )

        # Ask OpenAI to craft the short, friendly reply
        resp = self.client.chat.completions.create(
            model=self.model_id,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": message},
            ],
            temperature=0.6,
            max_tokens=80,
        )
        return resp.choices[0].message.content.strip()

        

## FAQ TOOL

In [3]:
from __future__ import annotations

import os
import re
from pathlib import Path
from typing import List, Tuple

from docx import Document
from smolagents import Tool, ToolCallingAgent
from smolagents.models import OpenAIModel
from openai import OpenAI



class FAQTool(Tool):
    name = "FAQTool"
    description = (
        "Answer customer FAQ-style questions strictly using the local file "
        "`general_faqs.docx` (same directory as this script). "
        "Use for queries about shipping, returns, refunds, warranties, hours, policies, pricing, etc."
    )

    inputs = {
        "question": {"type": "string", "description": "User's FAQ-style question."}
    }

    output_type = "string"

    def __init__(
        self,
        api_key: str,
        model_id: str = "gpt-4o-mini",
        docx_path: str | Path = "general_faqs.docx",
        max_snippets: int = 8
    ):
        """
        - Loads the docx once and keeps paragraphs in memory.
        - Selects top-k relevant paragraphs to stay within token limits.
        """
        super().__init__()
        self.client = OpenAI(api_key=api_key)
        self.model_id = model_id
        self.docx_path = Path(docx_path)
        self.max_snippets = max_snippets
        self.paragraphs = self._load_docx_paragraphs(self.docx_path)

        self.system_prompt = (
            "You are a customer support assistant. Answer ONLY using the provided DATA. "
            "If the DATA does not contain the answer, say you don't have enough information. "
            "Be concise and actionable."
        )

    # ---- File loading helpers ----
    @staticmethod
    def _load_docx_paragraphs(path: Path) -> List[str]:
        if not path.exists():
            raise FileNotFoundError(f"FAQ file not found: {path.resolve()}")
        doc = Document(str(path))
        paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
        # Merge tiny lines into bigger blocks (optional – keeps context coherent)
        merged: List[str] = []
        buf = []
        for p in paras:
            buf.append(p)
            if len(" ".join(buf)) > 300:  # simple merge threshold
                merged.append(" ".join(buf))
                buf = []
        if buf:
            merged.append(" ".join(buf))
        return merged

    # ---- Relevance selector: pick paragraphs most similar to the question ----
    @staticmethod
    def _keywordize(text: str) -> List[str]:
        tokens = re.findall(r"[a-z0-9]+", text.lower())
        stop = {
            "the","a","an","and","or","of","to","is","are","be","for","on","in",
            "at","by","with","from","as","that","this","it","we","you","your",
            "our","us","i"
        }
        return [t for t in tokens if t not in stop]

    def _score(self, paragraph: str, question: str) -> int:
        q = set(self._keywordize(question))
        p = set(self._keywordize(paragraph))
        return len(q & p)

    def _select_relevant_context(self, question: str) -> List[Tuple[int, str]]:
        scored = [(self._score(p, question), p) for p in self.paragraphs]
        scored.sort(key=lambda x: x[0], reverse=True)
        top = [(s, p) for s, p in scored[: self.max_snippets] if s > 0]
        # If nothing matched, still return a small sample to give the LLM something
        if not top and self.paragraphs:
            top = [(0, self.paragraphs[0])]
        # Attach indices to keep order stable (optional)
        return list(enumerate([p for _, p in top], start=1))

    # ---- Tool entrypoint ----
    def forward(self, question: str) -> str:
        # Lightweight trigger guard (optional; the agent usually routes to this tool)
        faq_cues = [
            "refund", "return", "shipping", "delivery", "warranty", "replacement",
            "order status", "track", "payment", "billing", "price", "discount",
            "hours", "open", "close", "support", "policy", "policies", "exchange",
            "cancel", "cancellation", "international", "customs", "duty", "tax"
        ]
        if not any(cue in question.lower() for cue in faq_cues):
            # Still proceed; the agent decided to call us. But we warn if it looks off.
            pass

        context_snippets = self._select_relevant_context(question)
        context_text = "\n\n".join(
            f"[Snippet {i}]\n{para}" for i, para in context_snippets
        )

        user_payload = (
            f"DATA:\n{context_text}\n\n"
            f"QUESTION:\n{question}\n\n"
            "INSTRUCTIONS:\n"
            "- Answer ONLY using DATA.\n"
            "- If not in DATA, say: 'I’m sorry, I don’t have that information.'\n"
            "- Keep it concise (2–5 sentences)."
        )

        resp = self.client.chat.completions.create(
            model=self.model_id,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": user_payload},
            ],
            temperature=0.2,
            max_tokens=300,
        )
        return resp.choices[0].message.content.strip()


## Answer Verification Tool

In [4]:
class VerifyAnswerTool(Tool):
    """
    Verifies if the user's question has been sufficiently answered
    by the latest tool output. If yes, returns 'FINALIZE: <answer>'.
    Otherwise returns 'CONTINUE: <reason>'.
    """
    name = "VerifyAnswerTool"
    description = (
        "Given the original user question and the latest candidate answer, "
        "decide if the question has been fully answered. If sufficient, return "
        "'FINALIZE: <answer>'. If not, return 'CONTINUE: <brief reason>'."
    )

    inputs = {
        "question": {"type": "string", "description": "Original user question."},
        "candidate_answer": {"type": "string", "description": "Latest tool/model output you're evaluating."}
    }

    output_type = "string"

    def __init__(self, api_key: str, model_id: str = "gpt-4o-mini"):
        super().__init__()
        self.client = OpenAI(api_key=api_key)
        self.model_id = model_id
        self.system_prompt = (
            "You are a strict verifier. Determine if the candidate answer is correct "
            "answers the user's question. If YES, respond exactly as:\n"
            "FINALIZE: <Answer as is>\n"
            "If NO, respond exactly as:\n"
            "CONTINUE: <one-sentence reason what is missing or unclear>\n"
            "Do not add any other text."
        )

    def forward(self, question: str, candidate_answer: str) -> str:
        msg = (
            f"QUESTION:\n{question}\n\n"
            f"CANDIDATE_ANSWER:\n{candidate_answer}\n\n"
            "Does CANDIDATE_ANSWER fully answer QUESTION?"
        )
        resp = self.client.chat.completions.create(
            model=self.model_id,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": msg},
            ],
            temperature=0.0,
            max_tokens=120,
        )
        return resp.choices[0].message.content.strip()

In [11]:
from smolagents import Tool, ToolCallingAgent
from smolagents.models import OpenAIModel
from smolagents import FinalAnswerTool, UserInputTool

# --- Create the OpenAI model ---
# You can also set OPENAI_API_KEY in your environment instead of passing api_key directly.
model = OpenAIModel(
    model_id="gpt-4o-mini",             # or "gpt-4o", "gpt-4.1", etc.
    api_key=OPENAI_API_KEY
)

# --- Register tools ---
tools = [
        Simple_Conversation_Tool(api_key=OPENAI_API_KEY, model_id="gpt-4o-mini"),
        FAQTool(api_key=OPENAI_API_KEY, model_id="gpt-4o-mini", docx_path="general_faqs.docx"),
        FinalAnswerTool(),
        UserInputTool()
    ]

INSTRUCTIONS = (
    "You are a tool-using customer support agent.\n"
    "Protocol for EVERY user message:\n"
    "1) Select exactly ONE domain tool (e.g., Simple_Conversation_Tool for greetings, "
    "   FAQTool for policy questions) and call it ONCE to produce a CANDIDATE_ANSWER.\n"
    "2) If the CANDIDATE_ANSWER fully addresses the user’s question:\n"
    "   - Call FinalAnswerTool with that answer and STOP.\n"
    "3) If the CANDIDATE_ANSWER is incomplete, unclear, or irrelevant:\n"
    "   - Call another domain tool that is most likely to improve the answer.\n"
    "   - OR, if clarification is absolutely required, call UserInputTool to ask the user.\n"
    "   - After refining, repeat step 2.\n"
    "4) If user question is beyond your capabilites answer, call UserInputTool "
    "   to request input from your customer support employee.\n"
    "Rules:\n"
    "- NEVER invent extra user messages or simulate a conversation with yourself.\n"
    "- Keep tool calls minimal — finalize as soon as you have a solid answer.\n"
    "- Always prefer short, clear, helpful answers.\n"
)

# --- Create the agent ---
agent = ToolCallingAgent(
        model=model,
        tools=tools,
        instructions=INSTRUCTIONS,
        max_steps=5  # domain tool -> verify -> maybe one more hop -> verify -> finalize
    )

# --- Run queries ---
query = input("User: ")
response = agent.run(query)

print("User:", query)
print("Agent:", response)



╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what can you do for me                                                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'Simple_Conversation_Tool' with arguments: {'message': 'What can you do for me?'}                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: I'm here to help with any questions or support you may need! Please feel free to share more details 
about what you're looking for.

[Step 1: Duration 2.52 seconds| Input tokens: 1,431 | Output tokens: 23]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "I'm here to help with any questions or support you may │
│ need! Please feel free to share more details about what you're looking for."}                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: I'm here to help with any questions or support you may need! Please feel free to share more details 
about what you're looking for.

Final answer: I'm here to help with any questions or support you may need! Please feel free to share more details 
about what you're looking for.

[Step 2: Duration 1.18 seconds| Input tokens: 2,958 | Output tokens: 62]

User: what can you do for me
Agent: I'm here to help with any questions or support you may need! Please feel free to share more details about what you're looking for.
